# STEP 1 - Grid Generation
Take a pre-set list of instructions and generate a 10x10 grid

In [13]:
def GridGen(obs):
    '''Takes in a list of obstructions and outputs a grid'''
    gen = [["□" for _ in range(10)] for _ in range(10)]
    for a in obs:
        list(a)
        gen[a[0]][a[1]]="■"
    return gen


# STEP 2 - Random Grid Generation
Generate a grid with 20 randomly placed obstacles on it, that are non-overlapping and do not overwrite (0,0) or (9,9)

In [14]:
def RandomGridGen():
    '''Generates a grid with 20 random obstacles'''
    import random
    randgrid = [["□" for _ in range(10)] for _ in range(10)]
    count = 0
    while count < 20:
        r1 = random.randint(0,9)
        r2 = random.randint(0,9)
        if (randgrid[r1][r2] != "■") and ((r1,r2) != (0,0)) and ((r1,r2) != (9,9)):
            randgrid[r1][r2] = "■"
            count += 1
    return randgrid

# STEP 3 - Print Grid Function
Takes in a grid and prints it line by line

In [15]:
def PrintGrid(grid):
    for i in grid:
        print(*i, sep="\t")
        print()
        print()
        
PrintGrid(RandomGridGen())

□	□	□	□	□	□	□	□	□	□


□	■	□	■	■	□	■	□	■	□


■	□	□	□	□	□	□	□	□	□


□	□	■	□	■	□	□	□	□	□


■	□	□	□	■	■	□	□	□	■


□	□	□	□	□	□	■	□	□	□


□	□	□	□	□	□	□	□	□	□


□	□	■	□	□	■	□	□	□	□


□	□	□	□	□	□	□	■	□	■


□	■	■	□	□	□	■	□	□	□




# STEP 4 - Possible Moves
Takes in a grid and a previous move and based on obstacle locations in that grid, outputs possible moves

In [16]:
def PossibleMoves(grid,prev):
    '''Takes in grid state and previous moves'''
    #if in centre, 8 moves possible
    #exceptions - side (value over 9 or under 0), OBS
    a,b = prev[0],prev[1]
    uu = ((a+1),(b+1))
    us = ((a+1),(b))
    ud = ((a+1),(b-1))
    su = ((a),(b+1))
    sd = ((a),(b-1))
    du = ((a-1),(b+1))
    ds = ((a-1),(b))
    dd = ((a-1),(b-1))
    moves = (uu,ud,du,dd,us,su,sd,ds)
    newmoves = list(moves)
    for m in moves:
        if m[0] > 9 or m[0] < 0 or m[1] > 9 or m[1] < 0 or grid[m[0]][m[1]]=="■":
            newmoves.remove(m)
    return newmoves


# STEP 5 - Obstacle Locations
With a similar functionality to Possible moves, this function takes a grid and a location and reports surrounding obstacle locations

In [17]:
def ObstacleLocations(grid,location):
    '''Checks surrounding places for obstacles'''
    a, b = location[0], location[1]    
    uu = ((a+1),(b+1))
    us = ((a+1),(b))
    ud = ((a+1),(b-1))
    su = ((a),(b+1))
    sd = ((a),(b-1))
    du = ((a-1),(b+1))
    ds = ((a-1),(b))
    dd = ((a-1),(b-1))
    moves = (uu,ud,du,dd,us,su,sd,ds)
    obstacles = []
    for m in moves:
        if (m[0] < 10) and (m[1] < 10) and (m[0] > -1) and (m[1] > -1):
            if grid[m[0]][m[1]] == "■":
                obstacles.append(m)
    
    return obstacles

# STEP 6 - Score Function
Takes in a grid and a move list and finds the best move based on distance from the start and endpoints (implementation of A* pathfinding algorithm)

In [18]:
def Score(grid,pmove):
    '''takes in a grid and move list and finds the best move based on distance from start (0,0) and end (9,9)'''
    from math import sqrt                                                                   #import square root
    movelist = PossibleMoves(grid,pmove)                                                    #get list of possible moves
    #FINISH THIS IF MOVELIST == NONE
    surrounding_obstacles = ObstacleLocations(grid,pmove)                                   #get list of surrounding obstacles
    best = 1000.0                                                                                #best score starts at 0
    bm = 0                                                                                  #best move is empty

    if movelist == []:                                                                      #when path is blocked
        print("No moves available")
        print(f"Remove obstacles from one of these positions:{surrounding_obstacles}")
        return None

    for i in movelist:                                                                      #when path not blocked - score each move and find best
        a,b = i[0],i[1]
        low = sqrt(a**2+b**2)
        sum_ab = sqrt((9)**2+(9)**2) - low
        if sum_ab < best:
            best = sum_ab
            bm = (a,b)
    return bm

grid = GridGen([(0,2),(0,3), (0,6), (0,7), (1,5), (1,8), (1,9), (2,1), (2,5), (3,4), (3,7), (4,4), (4,6), (5,6), (6,2), (7,1), (8,1), (8,5), (8,8), (9,6)])
print(Score(grid, (9,7)))

(9, 8)


# STEP 7 - Score Obstacle List
Takes in grid, previous move and obstacle list and outputs the ideal obstacle to remove to clear a path to the endpoint. Uses scoring system similar to the previous score function 

In [19]:
def ScoreObstacleList(grid,pmove,obstacle_list):
    '''takes in a grid and move list and finds the best move based on distance from start (0,0) and end (9,9)'''
    from math import sqrt                                                                   #import square root
    surrounding_obstacles = ObstacleLocations(grid,pmove)                                   #get list of surrounding obstacles
    best = 1000                                                                                #best score starts at 0
    bm = 0                                                                                  #best move is empty

    for i in obstacle_list:                                                                      #when path not blocked - score each move and find best
        a,b = i[0],i[1]
        low = sqrt(a**2+b**2)
        sum_ab = sqrt((9)**2+(9)**2) - low
        if sum_ab < best:
            best = sum_ab
            bm = (a,b)
    return bm

# STEP 8 - Pathfinding Function
Takes in a 10x10 grid and finds the best path from (0,0) to (9,9)

In [20]:
def Pathfinder1(obstacles):
    '''Takes in a grid and finds the best path'''
    grid = GridGen(obstacles)
    bmove = Score(grid,(0,0))
    best_path = [(0,0)]
    best_path.append(bmove)
    while bmove != (9,9):
        if len(best_path)>2:
            if best_path[-1] == best_path[-3]:
                break
            bmove = Score(grid, bmove)
            best_path.append(bmove)
            continue

        bmove = Score(grid,bmove)
        best_path.append(bmove)
    if (9,9) not in best_path:
        if best_path[-1] == best_path[-3]:
            a = best_path[-2]
            best_path = best_path[:-1]
            surr_obs = ObstacleLocations(grid,a)
            remove_this_obstacle = ScoreObstacleList(grid,a,surr_obs)
            return f'Pathfinding failed. Path: {best_path}. Remove this obstacle: {remove_this_obstacle} to continue'
        return 'path not found - unknown error'

    steps_taken = len(best_path)
    return best_path, f'Steps taken: {steps_taken}'

# STEP 9 - Random Grid Pathfinder
Uses the random grid generation function and finds the best path for that random grid. If path is blocked, suggests best obstacle to remove to clear the path

In [21]:
def Pathfinder2():
    '''Takes in a grid and finds the best path'''
    grid = RandomGridGen()
    bmove = Score(grid,(0,0))
    best_path = [(0,0)]
    best_path.append(bmove)
    while bmove != (9,9):
        if len(best_path)>2:
            if best_path[-1] == best_path[-3]:
                break
            bmove = Score(grid, bmove)
            best_path.append(bmove)
            continue

        bmove = Score(grid,bmove)
        best_path.append(bmove)
    if (9,9) not in best_path:
        if best_path[-1] == best_path[-3]:
            a = best_path[-2]
            best_path = best_path[:-1]
            surr_obs = ObstacleLocations(grid,a)
            remove_this_obstacle = ScoreObstacleList(grid,a,surr_obs)
            return f'Pathfinding failed. Path: {best_path}. Remove this obstacle: {remove_this_obstacle} to continue'
        return 'path not found - unknown error'

    steps_taken = len(best_path)
    return best_path, f'Steps taken: {steps_taken}'

# STEP 10 - Complete Pathfinding Function
Uses the random grid generation function to create a grid and if obstacles are present, allows the user to remove them.

In [31]:
def Pathfinder():
    '''Takes in a grid and finds the best path'''
    g = RandomGridGen()                                                                 #generate grid
    
    bmove = Score(g,(0,0))                                                              #
    best_path = [(0,0)]
    best_path.append(bmove)
    while bmove != (9,9):
        if len(best_path)>2:
            if best_path[-1] == best_path[-3]:
                break
            bmove = Score(g, bmove)
            best_path.append(bmove)
            continue
        
        if bmove == None:
            
            return 'pathfinding stopped at (0,0), obstacles surrounding start position'
        
        bmove = Score(g,bmove)
        best_path.append(bmove)
    if (9,9) not in best_path:
        if best_path[-1] == best_path[-3]:
            a = best_path[-2]
            best_path = best_path[:-1]
            surr_obs = ObstacleLocations(g,a)
            remove_this_obstacle = ScoreObstacleList(g,a,surr_obs)
            if type(remove_this_obstacle) != tuple:
                print("Error - processing failed")
                if remove_this_obstacle == 0:
                    print("obstacle returning 0 instead of tuple")
                return 'failure'
            print(f"Path blocked. Obstacle at {remove_this_obstacle} is recommended for removal (marked with '▩' below)")
            print(f"Path stopped at {best_path[-1]}. This location is marked with '🔶' below")
            print(f"best path is: {best_path}")
            print(f"surrounding obstacles are: {surr_obs}")
            g[best_path[-1][0]][best_path[-1][1]] = "🔶"
            g[remove_this_obstacle[0]][remove_this_obstacle[1]] = "▩"
            for m in best_path[:-1]:
                g[m[0]][m[1]] = "🔷"
            
        return 'failure', print(PrintGrid(g))

    steps_taken = len(best_path)
    for m in best_path:
        g[m[0]][m[1]] = "🔷"
    print(PrintGrid(g))
    return best_path, f'Steps taken: {steps_taken}'

Pathfinder()


🔷	□	□	□	□	□	□	□	□	□


□	🔷	□	■	□	■	□	■	□	■


□	□	🔷	□	□	□	□	□	□	□


■	□	□	🔷	■	□	■	■	□	□


□	□	□	□	🔷	■	□	■	□	■


□	□	□	□	□	🔷	■	□	□	□


□	□	□	□	□	□	🔷	□	□	□


□	□	■	■	□	□	□	🔷	□	□


□	■	□	□	□	□	□	□	🔷	□


□	■	□	■	□	■	■	■	□	🔷


None


([(0, 0),
  (1, 1),
  (2, 2),
  (3, 3),
  (4, 4),
  (5, 5),
  (6, 6),
  (7, 7),
  (8, 8),
  (9, 9)],
 'Steps taken: 10')